<a href="https://colab.research.google.com/github/amrios80/eda-jobs/blob/master/caoba/caoba_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projecto CAOBA (EDA)

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
**Datasets:** : (1) Demograficos, (2) Notas


**Business Context:** Se requiere conocer que variables mas inciden sobre la desercion academica.

**Restricciones:**
-

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

# **1. ETL**

**1.1 Cargue de los datasets:** Los datasets comprimidos se localizan en el repositorio.

In [3]:
demograficos_url="https://github.com/amrios80/eda-jobs/raw/refs/heads/main/caoba/Demograficos.csv.gz"
historicos_url="https://github.com/amrios80/eda-jobs/raw/refs/heads/main/caoba/HistoricoNotas.csv.gz"
demograficos_df = pd.read_csv(demograficos_url, sep=";", compression='gzip')
historicos_df=pd.read_csv(historicos_url, sep=";", compression="gzip")

<ipython-input-3-75dfc31c7cbb>:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  historicos_df=pd.read_csv(historicos_url, sep=";", compression="gzip")


 **1.2 Revision de dimensiones de los datasets.** Se observa: 55419 filas en demograficos, conteniendo los estados por todos los semestres de cada uno de los estudiantes.

In [4]:
print("dimensiones demo:",demograficos_df.shape)
print("dimensiones hist:", historicos_df.shape)

dimensiones demo: (55419, 33)
dimensiones hist: (418031, 12)


 **1.3 Encontrar el registro del último semestre.** Se encuentran 10201 ultimos semestres correspondientes cada uno a un solo estudiante.

In [6]:
def calculate_target(row):
    if (row['SITUACION_ACTUAL'] == 'EXCLUIDO NO RENOVACION DE MATRICULA' or row["SITUACION_ACTUAL"]=='RETIRO DEFINITIVO' or row["SITUACION_ACTUAL"]=='EXCLUIDO CANCELACION SEMESTRE' ) and  (row['CATEGORIA_ACTUAL'] != "CONTINUIDAD ACADEMICA - EGRESADO") :
        return 0
    else:
        return 1

demograficos_df["target"]=demograficos_df.apply(calculate_target, axis=1)
df=demograficos_df[ ~ (demograficos_df["NIVEL_FORMACION"]=="Especialización") ]

idx = df.groupby('CODIGO')['SEMESTRE'].idxmax()
max_semester_rows = demograficos_df.loc[idx]

df=max_semester_rows
df["YEARNACIMIENTO"]=pd.to_datetime(df["FECHANACIMIENTO"])
df['YEARNACIMIENTO'] = df['YEARNACIMIENTO'].dt.year
field="PROMEDIOGENERAL"
df[field] = pd.to_numeric(df[field].str.replace(',', '.'), errors='coerce')
field="PROMEDIOSEMESTRE"
df[field] = pd.to_numeric(df[field].str.replace(',', '.'), errors='coerce')


selected_cols=['PERIODO_ACAENICO', 'CODIGO', 'UNIDAD', 'CODIGOPROGRAMA','PROGRAMA',
       'SEMESTRE', 'PROMEDIOSEMESTRE', 'GENERO', 'FECHANACIMIENTO', 'ESTRATO',
       'CIUDADRESIDENCIA', 'DEPARTAMENTORESICEDENCIA', 'ESTADOCIVIL',
       'PROMEDIOGENERAL', 'SITUACION_ACTUAL', 'CATEGORIA_ACTUAL',
       'SITUACION_DEL_PERIODO', 'CATEGORIA_DEL_PERIODO', 'DISCAPACIDAD',
       'TIPO_DISCAPACIDAD', 'FECHA_MATRICULA', 'IDCIUDADNACIMIENTO',
       'IDDEPARTAMENTONACIMIENTO',
       'SISBEN', 'NIVELSISBEN', 'CANTIDAD',
       'PERIODO', 'EDAD2', 'EDAD', 'FACULTAD', 'NIVEL_FORMACION','YEARNACIMIENTO','target'
       ]

df=df[selected_cols]
df.to_csv("ultimo_semestre.csv",sep=";",index=False)
df.shape


<ipython-input-6-e048e64af83a>:14: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["YEARNACIMIENTO"]=pd.to_datetime(df["FECHANACIMIENTO"])


(10021, 33)

In [7]:
df["target"].value_counts()


,count
target,
1,8770
0,1251
